# Analyse de sentiments

C'est désormais un problème classique de machine learning. D'un côté, du texte, de l'autre une appréciation, le plus souvent binaire, positive ou négative mais qui pourrait être graduelle.

## Les données

On récupère les données depuis le site UCI [Sentiment Labelled Sentences Data Set](https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences). 

Il y a trois fichiers textes à importer et à concaténer pour former un dataset avec deux colonnes:
- Une "texte" comportant le contenu du commentaire
- Une "sentiment" comportant une note de 0 ou 1

#### Questions préalables: 
- Quelle est la taille du dataset ?
- Comment est distribuée la variable sentiment ?


#### Question 1: Combien y a t'il de mots dans l'ensemble du corpus ? Sont-ils "homogènes" ? 


### Import du module permettant le traitement de texte: nltk

In [31]:
import nltk
import os
from  module_nlp import *
from collections import Counter
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.model_selection import learning_curve
from sklearn.naive_bayes import CategoricalNB,BernoulliNB,ComplementNB,GaussianNB,MultinomialNB
from sklearn.model_selection import GridSearchCV, cross_val_score


#### Consigne 1: Récupérer le corpus de mots sans la ponctuation ni les stopwords

In [2]:
directory = 'sentiment labelled sentences/'

txts=[]
# iterate over files in

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)


In [3]:
# assign directory
directory = 'sentiment labelled sentences/'
dic={}
txts=[]
# iterate over files in

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
        # checking if it is a file

    if "labelled" in filename:
        name=filename.split("_")[0]

        if os.path.isfile(f):
            if f.endswith('.txt'):
                with open(f,'r', encoding="utf-8") as file:
                    txt = file.read()
                txts.append(txt)
                dic[name]={'txt':txt}


In [4]:
len(dic['amazon']['txt'])

58226

In [5]:
dic.keys()

dict_keys(['amazon', 'imdb', 'yelp'])

In [6]:
dic['mix']={
    'text':[],
    'site':[],
    'target':[],
    'clean':[],
    'clean1':[],
    'clean2':[]
}


for k,v in dic.items():
    if 'txt' in dic[k].keys():
        txt=dic[k]['txt']
        lst = txt.split("\n")

        dic[k]['tup']=[]
        dic[k]['clean_tup']=[]
        dic[k]['vec_tup']=[]
        dic[k]['target']=[]
        t_lst=[]

        for line in lst:
            if len(line.split("\t"))==2:
                t,n = tuple(line.split("\t"))

                dic[k]['target'].append(n)
                

                dic[k]['tup'].append((t,n))

                c1 =(clean(t))
                c2=(clean(t,2))
                dic[k]['clean_tup'].append((c1,n))

                dic['mix']['target'].append(n)
                dic['mix']['site'].append(k)
                dic['mix']['text'].append(t)

                dic['mix']['clean1'].append(c1)
                dic['mix']['clean2'].append(c2)
                

                t_lst.append(t)

            # dic[k]['clean_tup'].append((t,n))

    # dic[k]['tfidf'] = tfidf(t_lst)





In [7]:
df_a=tfidf(dic['mix']['text'])[1]
df_b=pd.DataFrame({
    'site1':dic['mix']['site'],
    'target':dic['mix']['target']
    })

df = pd.concat([df_a, df_b],axis=1)

In [8]:
df

,00,10,100,11,12,13,15,15g,15pm,17,...,yum,yummy,yun,z500a,zero,zillion,zombie,zombiez,site1,target
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,amazon,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,amazon,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,amazon,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,amazon,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,amazon,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,yelp,0
2996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,yelp,0
2997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,yelp,0
2998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,yelp,0


In [9]:
df['site1'].unique()

array(['amazon', 'imdb', 'yelp'], dtype=object)

In [55]:
c_lst=['text','clean1']

mods=[
    #KNeighborsClassifier(),
    LogisticRegression(),
    #CategoricalNB(class_prior=df.shape),
    BernoulliNB(),
    ComplementNB(),
    #GaussianNB(),
    MultinomialNB()
    ]
l=[]
for v in c_lst:
    l.extend(v*len(mods))

arrays = [
        mods * len(c_lst),
        c_lst * len(mods)
    ]

tuples = list(zip(*arrays))
    
rm=[] 
for mod in mods:
    rc=[]
    for c in c_lst:
        # print(mod, c)
        
        df_a=tfidf(dic['mix'][c])[1]
        df_b=pd.DataFrame({
            'site1':dic['mix']['site'],
            'target':dic['mix']['target']
            })

        df = pd.concat([df_a, df_b],axis=1)
        df = ohe(df,['site1'])
        
        X=df.drop('target',axis=1)
        y=df.target
        X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,train_size=0.8)

        
        clf = mod
        scores = cross_val_score(clf, X_train, y_train, cv=10, scoring = "accuracy")
        # print("Scores:", scores)
        print("Mean:", round(scores.mean(),3))
        # print("Standard Deviation:", round(scores.std()*100, 2))
        clf.fit(X_train,y_train)
        print("Test Score : ",round(clf.score(X_test,y_test),3))
        print("----")

        rm.append([round(scores.mean(),3),round(clf.score(X_test,y_test),3)])

    # rm.append(rc)

print(rm)

# res= pd.Series(rm,index = arrays)
# res

# results2 = pd.DataFrame({
# 'Model': ['KNN', 'Logistic Regression', 'Random Forest', 'MultinomialNB', 'GaussianNB'],
# 'Score': [ acc_knn, acc_logit, acc_rf, acc_mnb, acc_gnb]
# })
    
    # result_df2 = results2.sort_values(by='Score', ascending=False)
    # result_df2 = result_df2.set_index('Score')
    # result_df2
    # print('____')



Mean: 0.802
Test Score :  0.825
----
Mean: 0.802
Test Score :  0.798
----
Mean: 0.804
Test Score :  0.812
----
Mean: 0.798
Test Score :  0.805
----
Mean: 0.798
Test Score :  0.807
----
Mean: 0.799
Test Score :  0.798
----
Mean: 0.8
Test Score :  0.807
----
Mean: 0.799
Test Score :  0.797
----
[[0.802, 0.825], [0.802, 0.798], [0.804, 0.812], [0.798, 0.805], [0.798, 0.807], [0.799, 0.798], [0.8, 0.807], [0.799, 0.797]]


In [54]:
res= pd.DataFrame(rm,index = arrays,columns=['cv mean','test'])
res
# .sort_values('test',ascending=False)

# results2 = pd.DataFrame({
# 'Model': ['KNN', 'Logistic Regression', 'Random Forest', 'MultinomialNB', 'GaussianNB'],
# 'Score': [ acc_knn, acc_logit, acc_rf, acc_mnb, acc_gnb]
# })
    
#     result_df2 = results2.sort_values(by='Score', ascending=False)
#     result_df2 = result_df2.set_index('Score')
#     result_df2
#     print('____')

ValueError: all arrays must be same length

In [41]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,train_size=0.8)

params={
    'penalty':['l1','l2','elasticnet'],
    'C':[0.1,1,10,100,1000],
    #'solver':['liblinear','newton-cg']
    'max_iter':[1000]
}

clf=GridSearchCV(LogisticRegression(),params,cv=5)
clf.fit(X_train,y_train)
# print("Test Score : ",round(clf.score(X_test,y_test),3))


C:\Users\e_goy\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
50 fits failed out of a total of 75.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\e_goy\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\e_goy\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\e_goy\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_mo

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.1, 1, 10, 100, 1000], 'max_iter': [1000],
                         'penalty': ['l1', 'l2', 'elasticnet']})

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,train_size=0.8)

params={
    'penalty':['l1','l2','elasticnet'],
    'C':[0.1,1,10,100,1000],
    #'solver':['liblinear','newton-cg']
    'max_iter':[1000]
}

clf=GridSearchCV(LogisticRegression(),params,cv=5)
clf.fit(X_train,y_train)
# print("Test Score : ",round(clf.score(X_test,y_test),3))
# BernoulliNB()

In [ ]:
for i in dic:
    print(i)

amazon
imdb
yelp


In [ ]:
for k,v in dic.items():
    print(k)
    print(Counter(dic[k]['target']))

amazon
Counter({'0': 500, '1': 500})
imdb
Counter({'0': 500, '1': 500})
yelp
Counter({'1': 500, '0': 500})


In [ ]:
test=txts[1][:541]
test

'A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  \t0\nNot sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  \t0\nAttempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  \t0\nVery little music or anything to speak of.  \t0\nThe best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.  \t1\n'

In [ ]:
t=test.split("\n")[0]
t

'A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  \t0'

In [ ]:
tuple( t.split("\t"))

('A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  ',
 '0')

#### Question 2: Combien de mots restent-ils ? 

In [ ]:
clean_txts={
    'tokens':[],
    'txt':[]
}
mix={
    'tokens':[],
    'txt':""
}

for txt in txts:
    t=(clean(txt))
    clean_txts['tokens'].append(t)
    clean_txts['txt'].append(" ".join(t))

    mix['tokens'] += t
    mix['txt'] += " ".join(t)



In [ ]:
for i in clean_txts['tokens']:
    print ("nb token : ",len(i))
    print ("nb token unique: ",len(set(i)))
    print("---")

print ("all nb token : ",len(mix['tokens']))
print ("all nb token unique: ",len(set(mix['tokens'])))


nb token :  3422
nb token unique:  1103
---
nb token :  5322
nb token unique:  2116
---
nb token :  76
nb token unique:  48
---
nb token :  3950
nb token unique:  1302
---
all nb token :  12770
all nb token unique:  3588


## Exercice 1 : approche td-idf

La cible est la colonne *sentiment*, les deux autres colonnes sont les features. Il faudra utiliser les prétraitements [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html), [OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html), [TF-IDF](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html). L'un d'entre eux n'est pas nécessaire depuis la version [0.20.0](http://scikit-learn.org/stable/whats_new.html#sklearn-preprocessing) de *scikit-learn*.

#### Consigne 2: Utiliser OneHotEncoder ou CountVectorizer pour représenter les données sous forme d'une matrice contenant autant de colonnes que de mots dans le corpus

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer

#### Consigne 3: lancer un modèle de classification pour prédire les sentiments de 30% des observations



*   Séparer échantillon train / test
*   Entrainer un modèle de classification
*   Afficher la matrice de confusion
*   Calculer l'accuracy, la précision et le recall
*   Votre modèle est-il soumis à un overfitting ?

#### Consigne 4: Utiliser TF-IDF pour représenter les données et relancer un modèle de classification

## Exercice 2 : word2vec

On utilise l'approche [word2vec](https://en.wikipedia.org/wiki/Word2vec) du module [gensim](https://radimrehurek.com/gensim/models/word2vec.html) ou [spacy](https://spacy.io/usage/vectors-similarity).

#### Consigne 5: Utiliser Word2vec pour représenter les données et relancer un modèle de classification

## Exercice 3 : Comparer les deux approches

#### Consigne 6: Faire un graphique montrant les courbes ROC associées au deux modèles

#### Consigne 7: Faire un graphique montrant les courbes Precision-Recall associées aux deux modèles

### BONUS: reprendre l'étape de One_hot_encoding et ajouter une étape de réduction de dimension (type ACP)

In [ ]:
knight base